In [ ]:
#######################################################################################
# Author: Srijan Verma                                                              #
# School of Pharmacy                                                                #
# Sirimulla Research Group [http://www.sirimullaresearchgroup.com/]                 #
# The University of Texas at El Paso, TX, USA                                       #
# Last modified: 19/12/2019                                                         #
# Copyright (c) 2019 Srijan Verma and Sirimulla Research Group, under MIT license   #
#######################################################################################

In [1]:
import pandas as pd
from tqdm import tqdm_notebook
from glob import glob
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm_notebook
import pandas as pd
from glob import glob
import time
import os

In [17]:
df = pd.read_csv('../dataset/sub_inhib_2005_paper_data/3a4_inhib_data.csv')
df.head(2)

,Compound,Class,Set
0,Acetazolamide,P+,Training
1,Almotriptan,P+,Training


In [18]:
interesting_files = glob("../dataset/sub_inhib_2005_paper_data/*.csv")  
df = pd.concat((pd.read_csv(f, header = 0) for f in tqdm_notebook(interesting_files))) 
df_deduplicated = df.drop_duplicates() 
df_deduplicated.drop(['Class','Set'], axis=1, inplace=True)
df_deduplicated.reset_index(drop=True,inplace=True)

In [19]:
len(df_deduplicated)

1495

In [20]:
df_deduplicated.drop_duplicates(inplace=True) 
# df_deduplicated.drop_duplicates(subset=['Compound'],inplace=True)
df_deduplicated.reset_index(drop=True,inplace=True)
len(df_deduplicated)

/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


702

In [22]:
df_transformer = pd.read_csv('../dataset/transformer/from_drugbank_and_chembl_and_transformer_cleaned_v2.csv',index_col=0)
df_transformer_index = df_transformer.set_index('drug_name')

drug_not_found = []
smi = []
c = 0

for i in tqdm_notebook(range(len(df_deduplicated))):
    try:
        smi.append(df_transformer_index.loc[df_deduplicated['Compound'][i]]['smiles'])
    except:
        smi.append('-')
        c = c + 1
        drug_not_found.append(df_deduplicated['Compound'][i])

In [35]:
df_deduplicated['Smiles'] = smi
df_deduplicated.head(2)

/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Compound,Smiles
0,Albendazole,CCCSC1=CC2=C(NC(NC(=O)OC)=N2)C=C1
1,Almotriptan,CN(C)CCC1=CNC2=C1C=C(CS(=O)(=O)N3CCCC3)C=C2


In [23]:
c

176

In [67]:
# drug_not_found

In [28]:
get_missing_smi_from_drugBank = []
error_dr = []
for i in tqdm_notebook(range(len(drug_not_found))):
    
#     if missing_smi_drug_name[i] == 'Selenium Compounds' or missing_smi_drug_name[i] == 'Simvastatin And Ezetimibe' or missing_smi_drug_name[i] == 'Thiamine (vit B1)' or missing_smi_drug_name[i] == 'Sulfamethoxazole And Trimethoprim':
#         get_missing_smi_from_drugBank.append('check_again')
#         continue
        
#     else:
    try:
        url = 'https://www.drugbank.ca/unearth/q?utf8=%E2%9C%93&query='+ drug_not_found[i] + '&searcher=drugs'
        result = requests.get(url)
        c = result.content
        soup = BeautifulSoup(c)
        soup_string = str(soup)
        get_missing_smi_from_drugBank.append(soup_string[soup_string.find('>SMILES</dt>')+61:soup_string.find('</div>', soup_string.find('>SMILES</dt>')+40)])

    except:
        error_dr.append(drug_not_found[i])
        get_missing_smi_from_drugBank.append('-')
        pass
# str5 = soup_string[soup_string.find('>Protein binding<')+55:soup_string.find('>Metabolism<')-39]


In [31]:
for i in range(len(get_missing_smi_from_drugBank)):
    if 'href' in get_missing_smi_from_drugBank[i] or '<link' in get_missing_smi_from_drugBank[i]:
        get_missing_smi_from_drugBank[i] = '-'

In [64]:
# get_missing_smi_from_drugBank

In [ ]:
# //*[@id="Canonical-SMILES"]/div[1]/div/h4/span[2]
# //*[@id="Canonical-SMILES"]/div[2]/div[1]/p

In [60]:
from selenium import webdriver
driverpath = '../chrome/chromedriver'
driver = webdriver.Chrome(driverpath)

In [53]:
# driver.get('https://pubchem.ncbi.nlm.nih.gov/compound/' + drug_not_found[0])

In [43]:
# driver.get('https://pubchem.ncbi.nlm.nih.gov/compound/Avitriptan')

In [55]:
# driver.find_element_by_xpath('//*[@id="Canonical-SMILES"]/div[2]/div[1]/p').text

In [61]:
smi_from_pubmed = []
error_drug = []
for i in tqdm_notebook(range(len(drug_not_found))):

    try:
        driver.get('https://pubchem.ncbi.nlm.nih.gov/compound/' + drug_not_found[i])
        time.sleep(1.5)
        smi_from_pubmed.append(driver.find_element_by_xpath('//*[@id="Canonical-SMILES"]/div[2]/div[1]/p').text)
        time.sleep(0.5)
    except:
        smi_from_pubmed.append('-')
        error_drug.append(drug_not_found[i])
        pass


In [65]:
error_drug

[]

In [68]:
df_not_f_is_found = pd.DataFrame(drug_not_found,columns=['drug_name'])
df_not_f_is_found['smi_from_drugbank'] = get_missing_smi_from_drugBank
df_not_f_is_found['smi_from_pubmed'] = smi_from_pubmed
# df_not_f_is_found

In [73]:
df_deduplicated

,Compound,Smiles
0,Albendazole,CCCSC1=CC2=C(NC(NC(=O)OC)=N2)C=C1
1,Almotriptan,CN(C)CCC1=CNC2=C1C=C(CS(=O)(=O)N3CCCC3)C=C2
2,Alprenolol,CC(C)NC[C@H](O)COC1=CC=CC=C1CC=C
3,Amfetamine,C[C@@H](N)CC1=CC=CC=C1
4,Amiflamine,-
5,Aminopyrine,-
6,Amiodarone,CCCCC1=C(C(=O)C2=CC(I)=C(OCCN(CC)CC)C(I)=C2)C3...
7,Amodiaquine,CCN(CC)CC1=C(O)C=CC(NC2=C3C=CC(Cl)=CC3=NC=C2)=C1
8,Amoxapine,-
9,Amprenavir,CC(C)CN(C[C@@H](O)[C@H](CC1=CC=CC=C1)NC(=O)O[C...


In [82]:
df_not_f_is_found[df_not_f_is_found.drug_name == df_deduplicated['Compound'][4]]['smi_from_pubmed'].values[0]

'CC1=C(C=CC(=C1)N(C)C)CC(C)N'

In [80]:
for i in tqdm_notebook(range(len(df_deduplicated))):
    if df_deduplicated['Smiles'][i] == '-':
        df_deduplicated['Smiles'][i] = df_not_f_is_found[df_not_f_is_found.drug_name == df_deduplicated['Compound'][i]]['smi_from_pubmed'].values[0]

/Users/pankajverma/anaconda3/envs/seqcomhol/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [83]:
# df_deduplicated

In [84]:
df_deduplicated.to_csv('../dataset/sub_inhib_2005_paper_data/unique_smi_drug_names.csv')

In [2]:
df_deduplicated = pd.read_csv('../dataset/sub_inhib_2005_paper_data/unique_smi_drug_names.csv',index_col = 0)

In [3]:
df_deduplicated.head(2)

,Compound,Smiles
0,Albendazole,CCCSC1=CC2=C(NC(NC(=O)OC)=N2)C=C1
1,Almotriptan,CN(C)CCC1=CNC2=C1C=C(CS(=O)(=O)N3CCCC3)C=C2


In [6]:
df_deduplicated_index = df_deduplicated.set_index('Compound')

In [8]:
for csv in tqdm_notebook(glob('../dataset/sub_inhib_2005_paper_data/data_without_smiles/*.csv')):
    df = pd.read_csv(csv)
    smi = []
    target = []
    out = os.path.splitext(os.path.basename(csv))[0]
    
    for j in range(len(df)):
        smi.append(df_deduplicated_index.loc[df['Compound'][j]]['Smiles'])
        
        if (out.split('_')[1]) == 'sub' and df['Class'][j] == 'P+':
            target.append('Substrate') 
            
        elif (out.split('_')[1]) == 'inhib' and df['Class'][j] == 'P+':
            target.append('Inhibitor')
        
        else:
            target.append('-')
            
    df['Smiles'] = smi
    df['Type'] = target
    
    df.to_csv('../dataset/sub_inhib_2005_paper_data/data_with_smiles/'+ out +'.csv')